# Colab Start Section

Step 1 is to upload a copy of your private key. I am assuming your private key is named id_ed25519. If that isn't the case, you'll have to make manual changes.

In [2]:
# Change these of course to what is your email and your GitHub username
EMAIL = "pauleanderson@gmail.com"
USERNAME = "pauleanderson"
ORG = "anderson-fall-2024"

In [2]:
from google.colab import files
uploaded = files.upload()

Saving id_ed25519 to id_ed25519


In [3]:
%%bash -s "$EMAIL" "$USERNAME"

rm -rf /root/.ssh/*
mkdir /root/.ssh

mv id_ed25519 /root/.ssh/

chmod 600 /root/.ssh/id_ed25519

git config --global user.email "$1"
git config --global user.name "$2"

In [8]:
%%bash -s "$USERNAME" "$ORG"

eval "$(ssh-agent -s)"

ssh-add ~/.ssh/id_ed25519

ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

git clone git@github.com:$2/csc-487-$1.git

Agent pid 726


Identity added: /root/.ssh/id_ed25519 (pauleanderson@gmail.com)
# github.com:22 SSH-2.0-babeld-cd94d92b7
Cloning into 'csc-487-instructor'...


In [3]:
%load_ext autoreload
%autoreload 2


from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

import sys
sys.path.insert(0,f"csc-487-{USERNAME}")
sys.path.insert(0,"..")

import py487

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Assignment 6 - Attention is all you need

Paul E. Anderson

## Motivation

The vast majority of large language models today are based on or inspired by the attention/transformer architecture. You can find one of the original papers here - https://arxiv.org/abs/1706.03762.  

## Instructions:
In this assignment you will be coding classes that replicate some of the functionality from:
* csc-487-student/materials/Transformer_Towards_Data_Science.pdf

You must implement this code from numpy arrays and similar low level functions. I have added an option to my code called output_list which I use to easily convert back to tensors for display purposes. You may use tensors for displaying your code, but you should not use Pytorch to carry out the functionality needed to complete the assignment. If an object is given to you in Pytorch, you can use it of course. For example, you can use self.W_o. 

This code is inspirted by: https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb

Specifically, you are to re-implement the following using only Python and numpy/torch base functionality:

**You can use things like np.array or torch.tensor to make life easier, but you shouldn't use built in matrix multiplication, transpose, or similar functions.**
```
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
        def split_heads(self, x):
            batch_size, seq_length, d_model = x.size()
            return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output
```

**Notes: Please ignore the mask argument in your implementation.**

In [1]:
import torch

In [4]:
!pip install wget

Using legacy 'setup.py install' for wget, since package 'wheel' is not installed.
    Running setup.py install for wget ... done
You should consider upgrading via the '/Users/pander14/venv/bin/python3 -m pip install --upgrade pip' command.


In [7]:
import wget
url = 'http://nlp.stanford.edu/data/wordvecs/glove.6B.zip'
filename = wget.download(url,"/tmp/glove.6B.zip")
filename

100% [......................................................................] 862182753 / 862182753

'/tmp/glove.6B.zip'

In [9]:
!unzip -d /tmp/glove /tmp/glove.6B.zip

Archive:  /tmp/glove.6B.zip
  inflating: /tmp/glove/glove.6B.100d.txt  
  inflating: /tmp/glove/glove.6B.200d.txt  
  inflating: /tmp/glove/glove.6B.300d.txt  
  inflating: /tmp/glove/glove.6B.50d.txt  


## Loading the word vectors

In [4]:
import pandas as pd
glove = pd.read_csv('/tmp/glove/glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove_embedding = {key: val.values for key, val in glove.T.items()}
pd.Series(glove_embedding.keys()).sample(n=10)

318742         aftenblad
286557         saint-dié
252255              inss
344717             gejiu
279376              ōhia
139016    rhabdomyolysis
221820           jung-gu
30820           tightens
128164           legnano
184154       disasterous
dtype: object

**Example embeddings**

In [5]:
glove_embedding['queen']

array([-0.50045 , -0.70826 ,  0.55388 ,  0.673   ,  0.22486 ,  0.60281 ,
       -0.26194 ,  0.73872 , -0.65383 , -0.21606 , -0.33806 ,  0.24498 ,
       -0.51497 ,  0.8568  , -0.37199 , -0.58824 ,  0.30637 , -0.30668 ,
       -0.2187  ,  0.78369 , -0.61944 , -0.54925 ,  0.43067 , -0.027348,
        0.97574 ,  0.46169 ,  0.11486 , -0.99842 ,  1.0661  , -0.20819 ,
        0.53158 ,  0.40922 ,  1.0406  ,  0.24943 ,  0.18709 ,  0.41528 ,
       -0.95408 ,  0.36822 , -0.37948 , -0.6802  , -0.14578 , -0.20113 ,
        0.17113 , -0.55705 ,  0.7191  ,  0.070014, -0.23637 ,  0.49534 ,
        1.1576  , -0.05078 ,  0.25731 , -0.091052,  1.2663  ,  1.1047  ,
       -0.51584 , -2.0033  , -0.64821 ,  0.16417 ,  0.32935 ,  0.048484,
        0.18997 ,  0.66116 ,  0.080882,  0.3364  ,  0.22758 ,  0.1462  ,
       -0.51005 ,  0.63777 ,  0.47299 , -0.3282  ,  0.083899, -0.78547 ,
        0.099148,  0.039176,  0.27893 ,  0.11747 ,  0.57862 ,  0.043639,
       -0.15965 , -0.35304 , -0.048965, -0.32461 , 

In [6]:
glove_embedding['king']

array([-0.32307 , -0.87616 ,  0.21977 ,  0.25268 ,  0.22976 ,  0.7388  ,
       -0.37954 , -0.35307 , -0.84369 , -1.1113  , -0.30266 ,  0.33178 ,
       -0.25113 ,  0.30448 , -0.077491, -0.89815 ,  0.092496, -1.1407  ,
       -0.58324 ,  0.66869 , -0.23122 , -0.95855 ,  0.28262 , -0.078848,
        0.75315 ,  0.26584 ,  0.3422  , -0.33949 ,  0.95608 ,  0.065641,
        0.45747 ,  0.39835 ,  0.57965 ,  0.39267 , -0.21851 ,  0.58795 ,
       -0.55999 ,  0.63368 , -0.043983, -0.68731 , -0.37841 ,  0.38026 ,
        0.61641 , -0.88269 , -0.12346 , -0.37928 , -0.38318 ,  0.23868 ,
        0.6685  , -0.43321 , -0.11065 ,  0.081723,  1.1569  ,  0.78958 ,
       -0.21223 , -2.3211  , -0.67806 ,  0.44561 ,  0.65707 ,  0.1045  ,
        0.46217 ,  0.19912 ,  0.25802 ,  0.057194,  0.53443 , -0.43133 ,
       -0.34311 ,  0.59789 , -0.58417 ,  0.068995,  0.23944 , -0.85181 ,
        0.30379 , -0.34177 , -0.25746 , -0.031101, -0.16285 ,  0.45169 ,
       -0.91627 ,  0.64521 ,  0.73281 , -0.22752 , 

In [7]:
glove_embedding['prince']

array([-0.10462 , -0.44546 ,  0.73247 ,  0.33399 ,  0.74169 ,  0.26785 ,
       -0.13302 ,  0.048104, -0.58442 , -0.144   ,  0.042539,  0.91575 ,
       -0.27711 ,  0.025089,  0.54384 , -0.96325 , -0.15653 , -1.2196  ,
       -0.82354 ,  0.41574 , -0.5213  , -0.43856 ,  0.15554 , -0.33582 ,
       -0.26954 ,  0.28767 , -0.44932 ,  0.12631 ,  0.80588 ,  0.037442,
        0.083067, -0.16238 ,  0.66295 , -0.40002 , -0.23491 ,  0.32218 ,
        0.32082 ,  0.82943 , -0.73548 , -0.22967 ,  0.32303 ,  0.50304 ,
        0.42311 , -1.0475  ,  0.50004 , -0.26974 , -0.67122 ,  0.34976 ,
        0.84703 , -0.24704 ,  0.0171  ,  0.15045 ,  0.70515 ,  0.44578 ,
       -0.44401 , -2.1091  , -0.76305 ,  0.44046 ,  0.33391 ,  0.28334 ,
        0.17089 ,  0.1954  , -0.040206,  0.27536 ,  0.71581 , -0.16719 ,
       -0.12637 ,  0.53172 , -0.53269 ,  0.45923 ,  0.17436 , -1.0175  ,
        0.038111, -0.82138 , -0.15615 ,  0.35239 ,  0.31692 , -0.036803,
       -0.037231,  0.40949 ,  0.74874 ,  0.38309 , 

## Dataset
We are creating a simple dataset with two sentences. One of the sentences is shorter than the other, and so padding was added. 

In [8]:
d_k = len(glove_embedding["queen"])
num_heads = 5
print(f"d_k: {d_k}")

d_k: 100


### Generating our context-free word embeddings

We are using the glove context-free word embeddings. Because we want a multi-head attention, we are supplying ``num_heads`` copies of each embedding associated with each token in a sentence. For simplicity, I am only using two sentences. We are also padding this with zeros because the sentences are different lengths.

In [43]:
import torch
import numpy as np
sentence1 = "The king and the queen had a son who is the prince"
words1 = sentence1.split()
sentence2 = "The king of england is Charles"
words2 = sentence2.split()
n_tokens = max(len(words1),len(words2))

V1 = []
for i in range(n_tokens):
    if i < len(words1):
        V1.append(glove_embedding[words1[i].lower()].tolist()*num_heads)
    else:
        V1.append(np.zeros((d_k*num_heads,)))

V2 = []
for i in range(n_tokens):
    if i < len(words2):
        V2.append(glove_embedding[words2[i].lower()].tolist()*num_heads)
    else:
        V2.append(np.zeros((d_k*num_heads,)))
    
V = torch.tensor(np.array([V1,V2]),dtype=torch.float32)

In [44]:
V.shape

torch.Size([2, 12, 500])

In [45]:
V

tensor([[[-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
         [-0.3231, -0.8762,  0.2198,  ..., -0.5288,  0.1648, -0.9888],
         [-0.0720,  0.2313,  0.0237,  ..., -0.7189,  0.8689,  0.1954],
         ...,
         [-0.5426,  0.4148,  1.0322,  ..., -1.2969,  0.7622,  0.4635],
         [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
         [-0.1046, -0.4455,  0.7325,  ..., -0.6695, -0.2837, -1.2942]],

        [[-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
         [-0.3231, -0.8762,  0.2198,  ..., -0.5288,  0.1648, -0.9888],
         [-0.1529, -0.2428,  0.8984,  ..., -0.5910,  1.0039,  0.2066],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])

## Our attention object

In [46]:
# word embeddings times heads
attention = py487.attention.MultiHeadAttention(d_k*num_heads,num_heads)
attention.W_q
d_k*num_heads

500

## Examining our Shapes

In [47]:
V.shape

torch.Size([2, 12, 500])

In [48]:
torch.tensor(attention.split_heads(V,list_output=False)).shape

torch.Size([2, 5, 12, 100])

In [49]:
attention.split_heads(V,list_output=False)

array([[[[-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
           0.82779998,  0.27061999],
         [-0.32306999, -0.87616003,  0.21977   , ..., -0.52881002,
           0.16483   , -0.98878002],
         [-0.071953  ,  0.23127   ,  0.023731  , ..., -0.71894997,
           0.86894   ,  0.19539   ],
         ...,
         [-0.54263997,  0.41475999,  1.03219998, ..., -1.29690003,
           0.76217002,  0.46349001],
         [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
           0.82779998,  0.27061999],
         [-0.10462   , -0.44545999,  0.73246998, ..., -0.66948003,
          -0.28375   , -1.29419994]],

        [[-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
           0.82779998,  0.27061999],
         [-0.32306999, -0.87616003,  0.21977   , ..., -0.52881002,
           0.16483   , -0.98878002],
         [-0.071953  ,  0.23127   ,  0.023731  , ..., -0.71894997,
           0.86894   ,  0.19539   ],
         ...,
         [-0.54263997,  0.41475999

In [50]:
attention.combine_heads(attention.split_heads(V,list_output=True),list_output=False).shape

(2, 12, 500)

In [51]:
V

tensor([[[-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
         [-0.3231, -0.8762,  0.2198,  ..., -0.5288,  0.1648, -0.9888],
         [-0.0720,  0.2313,  0.0237,  ..., -0.7189,  0.8689,  0.1954],
         ...,
         [-0.5426,  0.4148,  1.0322,  ..., -1.2969,  0.7622,  0.4635],
         [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
         [-0.1046, -0.4455,  0.7325,  ..., -0.6695, -0.2837, -1.2942]],

        [[-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
         [-0.3231, -0.8762,  0.2198,  ..., -0.5288,  0.1648, -0.9888],
         [-0.1529, -0.2428,  0.8984,  ..., -0.5910,  1.0039,  0.2066],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])

In [52]:
attention.scaled_dot_product_attention(V,V,V,list_output=False)

array([[[-0.04169134, -0.2327853 ,  0.7080632 , ..., -0.16198099,
          0.8203707 ,  0.26874125],
        [-0.3198783 , -0.8696115 ,  0.22613053, ..., -0.5297519 ,
          0.15947372, -0.9885113 ],
        [-0.04153148,  0.19671911,  0.07274187, ..., -0.66336256,
          0.82956934,  0.18321793],
        ...,
        [-0.52202666,  0.39101028,  0.9926051 , ..., -1.2491548 ,
          0.7591393 ,  0.4512606 ],
        [-0.04169134, -0.2327853 ,  0.7080632 , ..., -0.16198099,
          0.8203707 ,  0.26874125],
        [-0.10619244, -0.4576801 ,  0.71165437, ..., -0.6663796 ,
         -0.27050242, -1.2821829 ]],

       [[-0.06321203, -0.23518014,  0.75859743, ..., -0.23100029,
          0.85375684,  0.26278245],
        [-0.32298926, -0.8756927 ,  0.21964346, ..., -0.5288897 ,
          0.1647979 , -0.98836374],
        [-0.15227912, -0.24188   ,  0.8967635 , ..., -0.5873809 ,
          1.0016779 ,  0.20760266],
        ...,
        [-0.1131205 , -0.05298417,  0.24620919, ..., -

In [53]:
Q2 = attention.split_heads(attention.W_q(V),list_output=False)
K2 = attention.split_heads(attention.W_k(V),list_output=False)
V2 = attention.split_heads(attention.W_v(V),list_output=False)
V2 = torch.tensor(V2)

In [54]:
V2.shape

torch.Size([2, 5, 12, 100])

In [55]:
torch.matmul(V2, V2.transpose(-2, -1)).shape

torch.Size([2, 5, 12, 12])

In [56]:
attention.scaled_dot_product_attention(V2,V2,V2,list_output=False).shape

(2, 5, 12, 100)

In [57]:
V.shape

torch.Size([2, 12, 500])

In [58]:
attention.forward(V,V,V,list_output=False)

tensor([[[-0.0963, -0.2358, -0.0071,  ..., -0.0491, -0.0180,  0.0432],
         [-0.1049, -0.2334, -0.0056,  ..., -0.0517, -0.0205,  0.0384],
         [-0.0961, -0.2358, -0.0078,  ..., -0.0489, -0.0169,  0.0416],
         ...,
         [-0.0998, -0.2343, -0.0020,  ..., -0.0500, -0.0193,  0.0408],
         [-0.0963, -0.2358, -0.0071,  ..., -0.0491, -0.0180,  0.0432],
         [-0.1115, -0.2294, -0.0007,  ..., -0.0513, -0.0236,  0.0367]],

        [[-0.0746, -0.1346, -0.0167,  ..., -0.0221,  0.0105,  0.0167],
         [-0.0753, -0.1315, -0.0190,  ..., -0.0295,  0.0095,  0.0142],
         [-0.0755, -0.1341, -0.0169,  ..., -0.0190,  0.0107,  0.0174],
         ...,
         [-0.0818, -0.1338, -0.0133,  ..., -0.0231,  0.0074,  0.0193],
         [-0.0818, -0.1338, -0.0133,  ..., -0.0231,  0.0074,  0.0193],
         [-0.0818, -0.1338, -0.0133,  ..., -0.0231,  0.0074,  0.0193]]],
       grad_fn=<ViewBackward0>)

In [59]:
V.transpose(-2, -1).shape

torch.Size([2, 500, 12])

## Outputs

In [60]:
# You got this! I output the code below in tensor format only to save output space. Please make sure your
# functions return lists to pass the tests.

In [61]:
attention.split_heads(V,list_output=False)

array([[[[-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
           0.82779998,  0.27061999],
         [-0.32306999, -0.87616003,  0.21977   , ..., -0.52881002,
           0.16483   , -0.98878002],
         [-0.071953  ,  0.23127   ,  0.023731  , ..., -0.71894997,
           0.86894   ,  0.19539   ],
         ...,
         [-0.54263997,  0.41475999,  1.03219998, ..., -1.29690003,
           0.76217002,  0.46349001],
         [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
           0.82779998,  0.27061999],
         [-0.10462   , -0.44545999,  0.73246998, ..., -0.66948003,
          -0.28375   , -1.29419994]],

        [[-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
           0.82779998,  0.27061999],
         [-0.32306999, -0.87616003,  0.21977   , ..., -0.52881002,
           0.16483   , -0.98878002],
         [-0.071953  ,  0.23127   ,  0.023731  , ..., -0.71894997,
           0.86894   ,  0.19539   ],
         ...,
         [-0.54263997,  0.41475999

In [65]:
attention.combine_heads(attention.split_heads(V,list_output=False),list_output=False)

array([[[-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
          0.82779998,  0.27061999],
        [-0.32306999, -0.87616003,  0.21977   , ..., -0.52881002,
          0.16483   , -0.98878002],
        [-0.071953  ,  0.23127   ,  0.023731  , ..., -0.71894997,
          0.86894   ,  0.19539   ],
        ...,
        [-0.54263997,  0.41475999,  1.03219998, ..., -1.29690003,
          0.76217002,  0.46349001],
        [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
          0.82779998,  0.27061999],
        [-0.10462   , -0.44545999,  0.73246998, ..., -0.66948003,
         -0.28375   , -1.29419994]],

       [[-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
          0.82779998,  0.27061999],
        [-0.32306999, -0.87616003,  0.21977   , ..., -0.52881002,
          0.16483   , -0.98878002],
        [-0.1529    , -0.24279   ,  0.89837003, ..., -0.59100002,
          1.00390005,  0.20664001],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [66]:
attention.scaled_dot_product_attention(V,V,V,list_output=False)

array([[[-0.04169134, -0.2327853 ,  0.7080632 , ..., -0.16198099,
          0.8203707 ,  0.26874125],
        [-0.3198783 , -0.8696115 ,  0.22613053, ..., -0.5297519 ,
          0.15947372, -0.9885113 ],
        [-0.04153148,  0.19671911,  0.07274187, ..., -0.66336256,
          0.82956934,  0.18321793],
        ...,
        [-0.52202666,  0.39101028,  0.9926051 , ..., -1.2491548 ,
          0.7591393 ,  0.4512606 ],
        [-0.04169134, -0.2327853 ,  0.7080632 , ..., -0.16198099,
          0.8203707 ,  0.26874125],
        [-0.10619244, -0.4576801 ,  0.71165437, ..., -0.6663796 ,
         -0.27050242, -1.2821829 ]],

       [[-0.06321203, -0.23518014,  0.75859743, ..., -0.23100029,
          0.85375684,  0.26278245],
        [-0.32298926, -0.8756927 ,  0.21964346, ..., -0.5288897 ,
          0.1647979 , -0.98836374],
        [-0.15227912, -0.24188   ,  0.8967635 , ..., -0.5873809 ,
          1.0016779 ,  0.20760266],
        ...,
        [-0.1131205 , -0.05298417,  0.24620919, ..., -

In [67]:
attention.forward(V,V,V,list_output=False)

tensor([[[-0.0963, -0.2358, -0.0071,  ..., -0.0491, -0.0180,  0.0432],
         [-0.1049, -0.2334, -0.0056,  ..., -0.0517, -0.0205,  0.0384],
         [-0.0961, -0.2358, -0.0078,  ..., -0.0489, -0.0169,  0.0416],
         ...,
         [-0.0998, -0.2343, -0.0020,  ..., -0.0500, -0.0193,  0.0408],
         [-0.0963, -0.2358, -0.0071,  ..., -0.0491, -0.0180,  0.0432],
         [-0.1115, -0.2294, -0.0007,  ..., -0.0513, -0.0236,  0.0367]],

        [[-0.0746, -0.1346, -0.0167,  ..., -0.0221,  0.0105,  0.0167],
         [-0.0753, -0.1315, -0.0190,  ..., -0.0295,  0.0095,  0.0142],
         [-0.0755, -0.1341, -0.0169,  ..., -0.0190,  0.0107,  0.0174],
         ...,
         [-0.0818, -0.1338, -0.0133,  ..., -0.0231,  0.0074,  0.0193],
         [-0.0818, -0.1338, -0.0133,  ..., -0.0231,  0.0074,  0.0193],
         [-0.0818, -0.1338, -0.0133,  ..., -0.0231,  0.0074,  0.0193]]],
       grad_fn=<ViewBackward0>)